In [65]:
import os
import cv2
import imutils
import easyocr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO

### Load Model


In [66]:
# Load a model
model = YOLO("D:/Coding/Projects/number-plate-recognition/runs/detect/train24/weights/best.onnx")

### Ambil Nama File

In [67]:
# Lokasi folder dengan file gambar
folder_path = "D:/Coding/Projects/number-plate-recognition/data"

# Mendapatkan semua nama file dalam folder
file_names = os.listdir(folder_path)

# Ekstrak nomor plat dari nama file
category_plat = [file.split('_')[0].split('.')[0] for file in file_names]
nomor_plat = [file.split('_')[1].split('.')[0] for file in file_names]

# Buat path lengkap untuk setiap file
file_paths = [os.path.join(folder_path, file) for file in file_names]

# Buat DataFrame pandas
data = {'Filename': file_names, "Category": category_plat, 'Nomor Plat': nomor_plat, 'Path': file_paths}
df = pd.DataFrame(data)

# Tampilkan DataFrame
df

,Filename,Category,Nomor Plat,Path
0,K1_AB5092XM.jpg,K1,AB5092XM,D:/Coding/Projects/number-plate-recognition/da...
1,K1_AD6579AKG.jpg,K1,AD6579AKG,D:/Coding/Projects/number-plate-recognition/da...
2,K1_G6853EQ.jpg,K1,G6853EQ,D:/Coding/Projects/number-plate-recognition/da...
3,K1_R2989RP.jpg,K1,R2989RP,D:/Coding/Projects/number-plate-recognition/da...
4,K2_AB2761HX.jpg,K2,AB2761HX,D:/Coding/Projects/number-plate-recognition/da...
5,K2_AB4652YY.jpg,K2,AB4652YY,D:/Coding/Projects/number-plate-recognition/da...
6,K2_AB5889FX.jpg,K2,AB5889FX,D:/Coding/Projects/number-plate-recognition/da...
7,K2_B6449EZH.jpg,K2,B6449EZH,D:/Coding/Projects/number-plate-recognition/da...
8,K3_AA8767FC.jpg,K3,AA8767FC,D:/Coding/Projects/number-plate-recognition/da...
9,K3_AB1091SJ.jpg,K3,AB1091SJ,D:/Coding/Projects/number-plate-recognition/da...


### Auto Detection & OCR

In [68]:
# Define Font
font = cv2.FONT_HERSHEY_SIMPLEX     # Jenis font
font_scale = 3                      # Skala font
font_thickness = 5                  # Ketebalan font
font_color = (0, 255, 0)            # Warna biru (BGR)

In [69]:
folder_path_result = 'D:/Coding/Projects/number-plate-recognition/code/main/results'
results_OCR = []
for index, row in df.iterrows():
    filename = row["Filename"]
    category = row["Category"]
    nomor_plat = row["Nomor Plat"]
    path = row["Path"]
    print(f'Proses Ke-{index+1} | Deteksi : {filename}')
    
    # Read image
    img = cv2.imread(path)
    img_size = img.shape[0:2]

    # Predict plat
    results = model(path, conf=0.5)
    
    # Predict plat
    results = results[0]
    x1, y1, x2, y2 = None, None, None, None
    
    # Draw rectangle
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        cropped_image = img[int(y1):int(y2), int(x1):int(x2)]
        
    # Melakukan text recognition dengan easy ocr
    reader = easyocr.Reader(['en'])
    resultOcr = reader.readtext(cropped_image)
    
    # Mengambil hasil recognition plat
    text_to_write = " ".join([res[1] for res in resultOcr])
    results_OCR.append(text_to_write)
    
    # Gambar rectangle
    imgDraw = cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), font_color, font_thickness)  # font_color adalah warna merah

    # Tulis kata
    imgDraw2 = cv2.putText(imgDraw, text_to_write, (int(x1), int(y1) - 10), font, font_scale, font_color, font_thickness)  # font_color adalah warna merah

    # Simpan gambar yang telah dimodifikasi
    cv2.imwrite(os.path.join(folder_path_result, filename), imgDraw2)
    # plt.imshow(cv2.cvtColor(imgDraw2, cv2.COLOR_BGR2RGB))
    
    print(f'Proses Ke-{index+1} Done...')
    print('----------------------------------------------\n')

Proses Ke-1 | Deteksi : K1_AB5092XM.jpg


Loading D:\Coding\Projects\number-plate-recognition\runs\detect\train24\weights\best.onnx for ONNX Runtime inference...

image 1/1 D:\Coding\Projects\number-plate-recognition\data\K1_AB5092XM.jpg: 640x640 1 plat-nomor, 127.8ms
Speed: 5.0ms preprocess, 127.8ms inference, 12.0ms postprocess per image at shape (1, 3, 640, 640)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Proses Ke-1 Done...
----------------------------------------------

Proses Ke-2 | Deteksi : K1_AD6579AKG.jpg



image 1/1 D:\Coding\Projects\number-plate-recognition\data\K1_AD6579AKG.jpg: 640x640 1 plat-nomor, 195.8ms
Speed: 5.0ms preprocess, 195.8ms inference, 78.0ms postprocess per image at shape (1, 3, 640, 640)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



Proses Ke-2 Done...
----------------------------------------------

Proses Ke-3 | Deteksi : K1_G6853EQ.jpg


...

Proses Ke-14 Done...
----------------------------------------------

Proses Ke-15 | Deteksi : K4_AB1918GG.jpg



image 1/1 D:\Coding\Projects\number-plate-recognition\data\K4_AB1918GG.jpg: 640x640 1 plat-nomor, 147.3ms
Speed: 5.6ms preprocess, 147.3ms inference, 17.0ms postprocess per image at shape (1, 3, 640, 640)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



Proses Ke-15 Done...
----------------------------------------------

Proses Ke-16 | Deteksi : K4_R1776EL.jpg


image 1/1 D:\Coding\Projects\number-plate-recognition\data\K4_R1776EL.jpg: 640x640 1 plat-nomor, 299.5ms
Speed: 5.0ms preprocess, 299.5ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Proses Ke-16 Done...
----------------------------------------------



In [70]:
results_OCR

['AB 5092.XM 05 . 27',
 'AD 6579 AKC OsaKhi 08 ,2 4 HELMET',
 "C 6853 Eq '4",
 'R   2989 RP 04 ,25',
 'B HX 28 2761 02',
 'AB Yy 4652',
 'AB 5889 FX 27 12',
 '8 6Ll9 EZH 09.27',
 'Aa" 8767 FC 01.24',
 "AB 1091 SJ 04'25",
 'OX AB 1780 25 0 |',
 'MAB 1869 | J] 12.24',
 'AA 1450 4j',
 'AB 1351 IR G5  2 8',
 'AB 1918 G6 12 27 HoNda Tugu 074520423',
 'R 1776 EL 08.27']

In [71]:
df["Prediction"] = results_OCR
df

,Filename,Category,Nomor Plat,Path,Prediction
0,K1_AB5092XM.jpg,K1,AB5092XM,D:/Coding/Projects/number-plate-recognition/da...,AB 5092.XM 05 . 27
1,K1_AD6579AKG.jpg,K1,AD6579AKG,D:/Coding/Projects/number-plate-recognition/da...,"AD 6579 AKC OsaKhi 08 ,2 4 HELMET"
2,K1_G6853EQ.jpg,K1,G6853EQ,D:/Coding/Projects/number-plate-recognition/da...,C 6853 Eq '4
3,K1_R2989RP.jpg,K1,R2989RP,D:/Coding/Projects/number-plate-recognition/da...,"R 2989 RP 04 ,25"
4,K2_AB2761HX.jpg,K2,AB2761HX,D:/Coding/Projects/number-plate-recognition/da...,B HX 28 2761 02
5,K2_AB4652YY.jpg,K2,AB4652YY,D:/Coding/Projects/number-plate-recognition/da...,AB Yy 4652
6,K2_AB5889FX.jpg,K2,AB5889FX,D:/Coding/Projects/number-plate-recognition/da...,AB 5889 FX 27 12
7,K2_B6449EZH.jpg,K2,B6449EZH,D:/Coding/Projects/number-plate-recognition/da...,8 6Ll9 EZH 09.27
8,K3_AA8767FC.jpg,K3,AA8767FC,D:/Coding/Projects/number-plate-recognition/da...,"Aa"" 8767 FC 01.24"
9,K3_AB1091SJ.jpg,K3,AB1091SJ,D:/Coding/Projects/number-plate-recognition/da...,AB 1091 SJ 04'25
